In [1]:
import glob
import pandas as pd
import numpy as np
# import scipy.sparse 
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from xgboost import XGBClassifier
import time

In [2]:
# path = './hw1_git/11775-hws/videos/*.mp4'
path = './hw1_git/11775-hws/asrs/*.txt'

filelist = []

for file in glob.glob(path):
    filelist.append(file)

In [128]:
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element)
    return result

In [205]:
text = []
for i in range(len(filelist)):
# for i in range(10):
    with open (filelist[i], "r") as myfile:
        data=myfile.readlines()
        data=concatenate_list_data(data)
    text.append(data)
    if i % 1000 == 0:
        print('{}th txt file reading...'.format(i))
    else: pass

0th txt file reading...
1000th txt file reading...
2000th txt file reading...


#### Vectorizer TDIF Vectorizer

In [226]:
###TF-IDF Vectorization
vectorize = TfidfVectorizer(stop_words="english",min_df=2,sublinear_tf=True)#, norm='l1')    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
norm_bow = vectorize.fit_transform(text).toarray()
norm_data = pd.DataFrame(norm_bow)
norm_data.head(3)
# bow = vectorize.fit_transform(text)#.toarray()

,0,1,2,3,4,5,6,7,8,9,...,3522,3523,3524,3525,3526,3527,3528,3529,3530,3531
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [243]:
vect = CountVectorizer(stop_words="english")
# vect = CountVectorizer()
bow = vect.fit_transform(text).toarray()
norm_bow = normalize(bow, norm = 'l1', axis=1)
norm_data = pd.DataFrame(norm_bow)
norm_data.head(3)

,0,1,2,3,4,5,6,7,8,9,...,6976,6977,6978,6979,6980,6981,6982,6983,6984,6985
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### ASRS Modeling...

In [244]:
# Making Video name
video_name_ind = []
for i in range(len(filelist)):
    import re
    match_front = re.search('asrs/', filelist[i])
    match_end = re.search('.txt', filelist[i])
    video_name_ind.append(filelist[i][match_front.end():match_end.start()])
    video_name = pd.DataFrame({'video': video_name_ind})
#     print(i, filelist[i][match_front.end():match_end.start()])


In [245]:
# Making features columns
k = norm_data.shape[1]
column_name = ['video']
for i in range(k):
    column_name.append('feature_{}'.format(i))

total_data = pd.concat([video_name, norm_data], axis = 1)
total_data.columns = column_name

In [246]:
total_data.head(3)

,video,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_6976,feature_6977,feature_6978,feature_6979,feature_6980,feature_6981,feature_6982,feature_6983,feature_6984,feature_6985
0,HVC3995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HVC3694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HVC3332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [247]:
train_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/train', sep = ' ', header = None)
valid_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/val', sep = ' ', header = None)
test_ind = pd.read_csv('./hw1_git/11775-hws/hw1_code/list/test.video', sep = ' ', header = None)

train_ind['Data'] = 'TRAIN'
valid_ind['Data'] = 'VALID'
test_ind[1] = 'UNK'
test_ind['Data'] = 'TEST'

train_ind.columns = ['video','target','Data']
valid_ind.columns = ['video','target','Data']
test_ind.columns = ['video','target','Data']

In [248]:
data_lable = pd.concat([train_ind, valid_ind, test_ind], axis = 0).reset_index().drop('index', axis = 1)
# data_lable['target_p001'] = 
data_lable['target_p001'] = data_lable['target']
data_lable['target_p002'] = data_lable['target']
data_lable['target_p003'] = data_lable['target']
data_lable['target_p001_10'] = 1
data_lable['target_p002_10'] = 1
data_lable['target_p003_10'] = 1

data_lable['target_p001'][data_lable['target'] != 'P001'] = 'Other'
data_lable['target_p002'][data_lable['target'] != 'P002'] = 'Other'
data_lable['target_p003'][data_lable['target'] != 'P003'] = 'Other'
data_lable['target_p001_10'][data_lable['target'] != 'P001'] = 0
data_lable['target_p002_10'][data_lable['target'] != 'P002'] = 0
data_lable['target_p003_10'][data_lable['target'] != 'P003'] = 0

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/di

In [249]:
total_mart = total_data.merge(data_lable, how = 'left', on = 'video')

In [250]:
train_mart = total_mart[total_mart['Data'] == 'TRAIN']
valid_mart = total_mart[total_mart['Data'] == 'VALID']
test_mart  = total_mart[total_mart['Data'] == 'TEST']

In [251]:
print(train_mart.shape, valid_mart.shape, test_mart.shape)

((640, 6995), (319, 6995), (1267, 6995))


#### Modeling....

In [252]:
train_mart.head(3)

,video,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_6984,feature_6985,target,Data,target_p001,target_p002,target_p003,target_p001_10,target_p002_10,target_p003_10
1,HVC3694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,TRAIN,Other,Other,Other,0,0,0
3,HVC711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,P002,TRAIN,Other,P002,Other,0,1,0
19,HVC3481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,TRAIN,Other,Other,Other,0,0,0


In [258]:
def modeling_ap_SVM(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = SVC(kernel=chi2_kernel, probability=True)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of SVM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_AdaB(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = AdaBoostClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of AdaBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_Boost(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = GradientBoostingClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of Boosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

def modeling_ap_xgb(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results

In [262]:
SVM_results_p001 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
SVM_results_p002 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
SVM_results_p003 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of SVM Modeling : 0.138298988342 seconds =====
0.019996622936136788
===== The time consuming of SVM Modeling : 0.145498991013 seconds =====
0.02821316614420063
===== The time consuming of SVM Modeling : 0.130504131317 seconds =====
0.16251671752433383


In [259]:
AdaB_results_p001 = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
AdaB_results_p002 = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
AdaB_results_p003 = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of AdaBoosting Modeling : 0.497128009796 seconds =====
0.0925553319919517
===== The time consuming of AdaBoosting Modeling : 0.978235006332 seconds =====
0.03202846975088968
===== The time consuming of AdaBoosting Modeling : 0.837021112442 seconds =====
0.10007616565866818


In [260]:
Boost_results_p001 = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Boost_results_p002 = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Boost_results_p003 = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of Boosting Modeling : 0.159568071365 seconds =====
0.09218981710331191
===== The time consuming of Boosting Modeling : 0.153877973557 seconds =====
0.031914893617021274
===== The time consuming of Boosting Modeling : 0.158764839172 seconds =====
0.09978535885773993


In [261]:
Xgb_results_p001 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Xgb_results_p003 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

===== The time consuming of XgBoosting Modeling : 0.156743049622 seconds =====
0.02287581699346405
===== The time consuming of XgBoosting Modeling : 0.0810389518738 seconds =====
0.030100334448160536
===== The time consuming of XgBoosting Modeling : 0.128482103348 seconds =====
0.07315576059883151


In [241]:
===== The time consuming of Modeling : 0.502795934677 seconds =====
0.02681992337164751
===== The time consuming of Modeling : 0.543972969055 seconds =====
0.03501945525291829
===== The time consuming of Modeling : 0.651535987854 seconds =====
0.08946728520577304

SyntaxError: invalid syntax (<ipython-input-241-bf364ea5633f>, line 1)

In [242]:
===== The time consuming of Modeling : 0.188441038132 seconds =====
0.051043338683788124
===== The time consuming of Modeling : 0.183010101318 seconds =====
0.03501945525291829
===== The time consuming of Modeling : 0.183886051178 seconds =====
0.09318639879554727

SyntaxError: invalid syntax (<ipython-input-242-cd144e8e4226>, line 1)

In [ ]:
===== The time consuming of Modeling : 0.162078142166 seconds =====
0.0219435736677116
===== The time consuming of Modeling : 0.0856039524078 seconds =====
0.0313588850174216
===== The time consuming of Modeling : 0.160410165787 seconds =====
0.1558041282822041